# Overview of Solvers I

## Goal
- Get an overview of various solvers in Ocean
- Understand the main purpose of each solver
- Get familiar with some basic solver parameters

## Abstract Sampler class
- All Ocean samplers inherit from the abstract class
- It has the main methods `.sample`, `.sample_ising`, `.sample_qubo`, etc.
- We would like to focus on using `.sample` method with all solvers

In [ ]:
from dimod import Sampler

Sampler?

## Problem
To focus on samplers, we are going to create a simple BQM problem that we will solve using different solvers.

In [ ]:
from dimod import BinaryQuadraticModel, to_networkx_graph
bqm = BinaryQuadraticModel('SPIN')
bqm.add_variable(0, -1)
bqm.add_variable(1, -1)
bqm.add_variable(4, -1)
bqm.add_variable(5, -1)
bqm.add_interaction(0, 4, 1.0)
bqm.add_interaction(0, 5, 1.0)
bqm.add_interaction(1, 4, 1.0)
bqm.add_interaction(1, 5, 1.0)

import networkx as nx

nx.draw(to_networkx_graph(bqm))

## ExactSolver
- Mainly for debugging purposes
- Can solve problems with up to 20 variables (or more) depending on the system

In [ ]:
from dimod import ExactSolver

solver = ExactSolver()
response = solver.sample(bqm)
print(response.truncate(10))

## Simulated Annealing
- Historically was the main benchmark against QPU due to the similar underlying principles
- Can be a good solver/sampler for small problems
- A great tool for debugging purposes
- Can be slow for large, dense problems

In [ ]:
from neal import SimulatedAnnealingSampler

sampler = SimulatedAnnealingSampler()

response = sampler.sample(bqm, num_reads=10,
                          num_sweeps=1000,
                          initial_states=None,
                          beta_range=[10, 100])
print(response)

In [ ]:
sampler.sample?

In [ ]:
from dwave.system import DWaveSampler

sampler = DWaveSampler(solver=dict(topology__type='chimera'))

response = sampler.sample(
    bqm, num_reads=10,
    annealing_time=10,
    auto_scale=False,
    answer_mode='raw'
    )
print(response)

In [ ]:
sampler.sample?

## Some important parameters of the QPU
- num_reads=10
- annealing_time=10
- auto_scale: True or False

## LeapHybridSampler
- The most flexible solver 
- Can solve large, dense problems efficiently using classical and quantum resources
- 20,000 variable fully connected (~200M biases)
- 1 million variables with at most 200M biases
- Only one parameter - time limit (the minimum time limit is chosen by default)

In [ ]:
from dwave.system import LeapHybridSampler

sampler = LeapHybridSampler()
print(sampler.properties)

response = sampler.sample(
    bqm, time_limit=3,
    )
print(response)